In [ ]:
import requests
import json
from rdflib import Graph, Literal, Namespace, URIRef, plugin
from rdflib.namespace import RDF, RDFS
from rdflib.serializer import Serializer
import numpy as np

Final_RDF=""
movies_queries=[]

n=int(input("Enter the number of movies to scrape"))
for k in range(n):
    rep=input("Enter the name of the movie number "+str(k+1)+" : ")
    movies_queries.append(rep.lower())

for k in range(len(movies_queries)):
    # Define the API endpoint and parameters
    modified_title=""
    modified_title_Bis=movies_queries[k].split(" ")
    for k in range(len(modified_title_Bis)-1):
        modified_title+=modified_title_Bis[k]+"+"
    modified_title+=modified_title_Bis[-1]
        
    api_endpoint = "http://www.omdbapi.com/?t="+modified_title+"&apikey=deb918e7"
    api_params = {"format": "json"}

    # Make the API request and get the JSON data
    response = requests.get(api_endpoint, params=api_params)
    data = json.loads(response.content.decode("utf-8"))
    data = {k: v for k, v in data.items() if k in ["Title","Genre", "Year", "Director", "Writer", "Actors", "Language", "Country","Released","DVD"]}

    data_Directors = data["Director"].split(", ")
    data_Actors = data["Actors"].split(", ")
    data_Writers = data["Writer"].split(", ")

    data_Genre = data["Genre"].split(", ")
    data_Language = data["Language"].split(", ")
    data_Country = data["Country"].split(", ")
    data_Theater=["KINEPOLIS_AMPHITHEATRE","PATHE-GAUMONT_AQUABOULEVARD","UGC_LA_DEFENSE"]
    
    api_Theater_endpoint = "https://data.culture.gouv.fr/api/records/1.0/search/?dataset=etablissements-cinematographiques&q=&rows=225&facet=region_administrative&facet=genre&facet=multiplexe&facet=zone_de_la_commune"
    api_Theater_params = {"format": "json"}

    # Make the API request and get the JSON data
    response_Theater = requests.get(api_Theater_endpoint, params=api_Theater_params)
    data_Theater_api = json.loads(response_Theater.content.decode("utf-8"))

    Theaters_Bis=[]

    for k in range(len(data_Theater_api["records"])):
        Theaters_Bis.append(data_Theater_api["records"][k]["fields"]["nom"])
    Theaters = [name for name in Theaters_Bis if name.startswith(("UGC", "MK", "Pathe"))]
    
    data_Theater.extend(Theaters)
    
    #Actors Section
    for k in range(len(data_Actors)):
        rep_rdf="""\n
    <!-- http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Actor} -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Actor}">
        <rdf:type rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#Actor"/>
        <isActorOf rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Title}"/>
        <age rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">{Age}</age>
        <name>{Name}</name>
        <nationality>American</nationality>
    </owl:NamedIndividual>\n
        """
        rep_rdf=rep_rdf.format(Actor=data_Actors[k].replace(" ","_"),Title=data["Title"].replace(" ","_"),Age=np.random.randint(35,50),Name=data_Actors[k])
        Final_RDF+=rep_rdf
        
    #Directors Section
    for k in range(len(data_Directors)):
        rep_rdf="""\n
    <!-- http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Director} -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Director}">
        <rdf:type rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#Director"/>
        <isDirectorOf rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Title}"/>
        <age rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">{Age}</age>
        <name>{Name}</name>
        <nationality>American</nationality>
    </owl:NamedIndividual>\n
        """
        rep_rdf=rep_rdf.format(Director=data_Directors[k].replace(" ","_"),Title=data["Title"].replace(" ","_"),Age=np.random.randint(35,50),Name=data_Directors[k])
        Final_RDF+=rep_rdf
        
    #Writers Section
    for k in range(len(data_Writers)):
        rep_rdf="""\n
    <!-- http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Writer} -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Writer}">
        <rdf:type rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#Writer"/>
        <isWriterOf rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{Title}"/>
        <age rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">{Age}</age>
        <name>{Name}</name>
        <nationality>American</nationality>
    </owl:NamedIndividual>\n
        """
        rep_rdf=rep_rdf.format(Writer=data_Writers[k].replace(" ","_"),Title=data["Title"].replace(" ","_"),Age=np.random.randint(35,50),Name=data_Writers[k])
        Final_RDF+=rep_rdf
        
    #Movie Section
    n1=np.random.randint(0,len(data_Theater))
    rep_rdf="""\n
    <!-- http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{title} -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{title}">
        <rdf:type rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#Movie"/>\n"""
    for k in range(len(data_Actors)):
        rep_rdf+='        <hasActor rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#'+data_Actors[k].replace(" ","_")+'"/>\n'
    for k in range(len(data_Directors)):
        rep_rdf+='        <hasDirector rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#'+data_Directors[k].replace(" ","_")+'"/>\n'
    for k in range(len(data_Genre)):
        rep_rdf+='        <hasGenre rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#'+data_Genre[k]+'"/>\n'
    for k in range(len(data_Writers)):
        rep_rdf+='        <hasWriter rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#'+data_Writers[k].replace(" ","_")+'"/>\n'
    rep_rdf+='        <isPlayedIn rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#'+data_Theater[n1].replace(" ","_")+'"/>'
    for k in range(len(data_Country)):
        rep_rdf+='        <country>'+data_Country[k].replace(" ","_")+'</country>\n'
    for k in range(len(data_Language)):
        rep_rdf+='        <language>'+data_Language[k].replace(" ","_")+'</language>\n'
    rep_rdf+='        <title>'+data["Title"]+'</title>\n'
    rep_rdf+='        <year rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">'+data["Year"]+'</year>\n'
    rep_rdf+='        <from_date>'+data["Released"]+'</from_date>\n'
    rep_rdf+='        <to_date>'+data["DVD"]+'</to_date>\n'
    rep_rdf+='    </owl:NamedIndividual>\n'
    
    #Theater Section
    rep_rdf+="""
        


    <!-- http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{TheaterName} -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{TheaterName}">
        <rdf:type rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#Theater"/>
        <isPlaying rdf:resource="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#{title}"/>
        <theater_name>{TheaterName}</theater_name>
    </owl:NamedIndividual>
    """
    
    rep_rdf=rep_rdf.format(title=data["Title"].replace(" ","_"),TheaterName=data_Theater[n1].replace(" ","_"))
    Final_RDF+=rep_rdf

with open("Movies_DB.rdf", "w") as f:
    f.write(Final_RDF)    

# Ouvrir le fichier en mode écriture
chemin = r'C:\Users\emine\Documents\Ecole\A4\S8\Web Datamining & Semantics\Final Project\Part I - Ontology.owl'
fichier = open(chemin, 'r')
contenu = fichier.read()

# Trouver l'emplacement de la balise </rdf:RDF>
position = contenu.find('</rdf:RDF>')

# Insérer le texte avant la balise
contenu_modifie = contenu[:position] + Final_RDF + contenu[position:]

# Écrire la chaîne modifiée dans le fichier
fichier = open(chemin, 'w')
fichier.write(contenu_modifie)
fichier.close()

print(Final_RDF)

# **Affichage du contenu du fichier**

In [4]:
chemin = r'C:\Users\emine\Documents\Ecole\A4\S8\Web Datamining & Semantics\Final Project\Part I - Ontology.owl'
fichier = open(chemin, 'r')
contenu = fichier.read()
print(contenu)
fichier.close()

<?xml version="1.0"?>
<rdf:RDF xmlns="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#"
     xml:base="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7"
     xmlns:owl="http://www.w3.org/2002/07/owl#"
     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
     xmlns:xml="http://www.w3.org/XML/1998/namespace"
     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
    <owl:Ontology rdf:about="http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7">
        <owl:imports rdf:resource="http://www.w3.org/2006/vcard/ns"/>
    </owl:Ontology>
    


    <!-- 
    ///////////////////////////////////////////////////////////////////////////////////////
    //
    // Object Properties
    //
    ///////////////////////////////////////////////////////////////////////////////////////
     -->

    


    <!-- http://www.semanticweb.org/emine/ontologies/2023/2/untitled-ontology-7#ha